In [4]:
%load_ext autoreload
%autoreload 2

In [15]:
import day16

day16.main("test.txt", debug=True)

Valve(name='AA', flowrate=0, tunnels=['DD', 'II', 'BB'])
Valve(name='BB', flowrate=13, tunnels=['CC', 'AA'])
Valve(name='CC', flowrate=2, tunnels=['DD', 'BB'])
Valve(name='DD', flowrate=20, tunnels=['CC', 'AA', 'EE'])
Valve(name='EE', flowrate=3, tunnels=['FF', 'DD'])
Valve(name='FF', flowrate=0, tunnels=['EE', 'GG'])
Valve(name='GG', flowrate=0, tunnels=['FF', 'HH'])
Valve(name='HH', flowrate=22, tunnels=['GG'])
Valve(name='II', flowrate=0, tunnels=['AA', 'JJ'])
Valve(name='JJ', flowrate=21, tunnels=['II'])

  885 Step(valve='BB', minute=6, openvalves={'DD': 2, 'BB': 5}) BB {'DD': 2, 'BB': 5}
  885 Step(valve='CC', minute=7, openvalves={'DD': 2, 'BB': 5}) CC {'DD': 2, 'BB': 5}
  885 Step(valve='AA', minute=7, openvalves={'DD': 2, 'BB': 5}) AA {'DD': 2, 'BB': 5}
  931 Step(valve='CC', minute=8, openvalves={'DD': 2, 'BB': 5, 'CC': 7}) CC {'DD': 2, 'BB': 5, 'CC': 7}
  885 Step(valve='DD', minute=8, openvalves={'DD': 2, 'BB': 5}) DD {'DD': 2, 'BB': 5}
  885 Step(valve='II', minute=8, open